In [ ]:
%matplotlib inline 
import pandas as pd
import numpy as np
import pandas_profiling as ppv
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [ ]:
ctr_revenue = pd.read_csv('abtest_example_ctr.csv')

In [ ]:
# ctr_revenue_copy.date = ctr_revenue_copy.date.to_datetime()
ctr_revenue['date'] = pd.to_datetime(ctr_revenue['date'])

In [ ]:
ppv.ProfileReport(ctr_revenue)

# Checking problem in dataset

There are missing data in column 'userid'

In [ ]:
for column in ctr_revenue.columns.values:
    print column, ctr_revenue[[column]].isnull().values.any()

In [ ]:
ctr_revenue.shape

Is there any user id falling into both test and control groups - any mixed users?

In [ ]:
userid_distinctgroups = ctr_revenue.groupby('userid')['groups'].nunique().reset_index(name = 'number_of_distinct_groups')
user_multigroup = userid_distinctgroups[userid_distinctgroups.number_of_distinct_groups > 1]
print user_multigroup.count()
# print user_multigroup

Any userid has more than 1 device id? - user with multiple devices

In [ ]:
userid_numOfDevices = ctr_revenue.groupby('userid')['deviceid'].nunique().reset_index(name = 'number_of_devices')
user_multiDevice = userid_numOfDevices[userid_numOfDevices.number_of_devices > 1]
print user_multiDevice.count()
# print user_multiDevice

Any device id has more than 1 user id?

In [ ]:
deviceid_NumOfUserid = ctr_revenue.groupby('deviceid')['userid'].nunique().reset_index(name = 'numner_of_userid')
device_multiUser =  deviceid_NumOfUserid[deviceid_NumOfUserid.numner_of_userid > 1]
# print device_multiUser
print device_multiUser.count()

In [ ]:
ctr_revenue_copy = ctr_revenue.copy()

create dummy variable - if any problems above, 1 else 0

In [ ]:
ctr_revenue_copy['pb_missing_userid'] = ctr_revenue.userid.isnull().values * 1

In [ ]:
user_multigroup_idSet = set(user_multigroup.userid.values)
ctr_revenue_copy['pb_userid_mix'] = ctr_revenue_copy['userid'].apply(lambda x: x in user_multigroup_idSet) * 1

In [ ]:
user_multiDevice_idSet = set(user_multiDevice.userid.values)
ctr_revenue_copy['pb_userid_mulD'] = ctr_revenue_copy['userid'].apply(lambda x: x in user_multiDevice_idSet) * 1

In [ ]:
device_multiUser_idSet = set(device_multiUser.deviceid.values)
ctr_revenue_copy['pb_deviceid_mulU'] = ctr_revenue_copy['deviceid'].apply(lambda x: x in device_multiUser_idSet) * 1

For simplicity, I create a combined column 1/0 if there is any problems. In real projects, 
I will do this separately for each problem

In [ ]:
ctr_revenue_copy['pb_all'] = ctr_revenue_copy.apply(lambda x: max(x['pb_missing_userid'], x['pb_userid_mix'], x['pb_userid_mulD'], x['pb_deviceid_mulU']), axis=1)

In [ ]:
fig, axes = plt.subplots(nrows = 3, ncols = 2, figsize = (10,10))
ctr_revenue_copy.groupby('pb_all')['userid'].nunique('userid')\
    .reset_index(name = 'number_of_n_userid').plot.bar(x = 'pb_all', y = 'number_of_n_userid', ax = axes[0][0])
ctr_revenue_copy.groupby('pb_userid_mix')['userid'].nunique('userid')\
    .reset_index(name = 'number_of_n_userid').plot.bar(x = 'pb_userid_mix', y = 'number_of_n_userid', ax = axes[0][1])
ctr_revenue_copy.groupby('pb_userid_mulD')['userid'].nunique('userid')\
    .reset_index(name = 'number_of_n_userid').plot.bar(x = 'pb_userid_mulD', y = 'number_of_n_userid', ax = axes[1][0])
ctr_revenue_copy.groupby('pb_deviceid_mulU')['userid'].nunique('userid')\
    .reset_index(name = 'number_of_n_userid').plot.bar(x = 'pb_deviceid_mulU', y = 'number_of_n_userid', ax = axes[1][1])
ctr_revenue_copy.groupby('pb_missing_userid')['userid'].nunique('userid')\
    .reset_index(name = 'number_of_n_userid').plot.bar(x = 'pb_missing_userid', y = 'number_of_n_userid', ax = axes[2][0])


In [ ]:
ctr_revenue_logit = pd.get_dummies(ctr_revenue_copy, prefix_sep='_', drop_first=True)

http://www.kellieottoboni.com/posts/2017/07/logistic-regression-python/ 
1. sklearn doesn't have estimated coefficients and model fit statistics. 
2. There is no way to switch off regularization in scikit-learn

So, I am using statsmodels.api (it can add intercept) I tried statsmodels.formula.api which can't add intercept.

In [ ]:
# import statsmodels.formula.api as sm
import statsmodels.api as sm
X = sm.add_constant(ctr_revenue_logit[['country_CN', 'country_GB', 'country_US', 'groups_treatment', 'device_Ios', 'device_Other', 'device_Web', 'views', 'clicks', 'revenue']])
y = ctr_revenue_logit[['pb_all']]
model = sm.Logit(y, X)
result = model.fit()

In [ ]:
result.summary()

Some country and device have significant result

# Deep dive into problems by country/devices

In [ ]:
# CA, CN and US have a bit more missing userid than GB
ctr_revenue_copy.groupby('country')['pb_missing_userid'].mean().reset_index(name = 'mean_pb_missing_userid')

In [ ]:
# CA has more mix userid. something wrong with experiment in CA
ctr_revenue_copy.groupby('country')['pb_userid_mix'].mean().reset_index(name = 'mean_pb_userid_mix').plot.bar(x = 'country')

In [ ]:
# all countries have similiar percentage of users with multiple devices
ctr_revenue_copy.groupby('country')['pb_userid_mulD'].mean().reset_index(name = 'mean_pb_userid_mulD').plot.bar(x = 'country')

In [ ]:
# all countries have similiar percentage of devices multiple users
ctr_revenue_copy.groupby('country')['pb_deviceid_mulU'].mean().reset_index(name = 'mean_pb_deviceid_mulU').plot.bar(x = 'country')

In [ ]:
# IOS and Other have more missing userid than Android and Web
ctr_revenue_copy.groupby('device')['pb_missing_userid'].mean().reset_index(name = 'mean_pb_missing_userid')

In [ ]:
# plotting
# android and web have more missing userid than ios and other
ctr_revenue_copy.groupby('device')['pb_missing_userid'].mean().reset_index(name = 'mean_pb_missing_userid').plot.bar(x='device')

In [ ]:
# IOS and Other have more missing userid than Android and Web
ctr_revenue_copy.groupby('device')['pb_userid_mix'].mean().reset_index(name = 'mean_pb_missing_userid')

In [ ]:
# plotting
# other and web have more mix userid than android and ios
ctr_revenue_copy.groupby('device')['pb_userid_mix'].mean().reset_index(name = 'mean_pb_userid_mix').plot.bar(x='device')

In [ ]:
# plotting
# other have more devices with multiple users than Android, ios and web
ctr_revenue_copy.groupby('device')['pb_deviceid_mulU'].mean().reset_index(name = 'mean_pb_deviceid_mulU').plot.bar(x='device')

In [ ]:
# IOS and Other have more missing userid than Android and Web
ctr_revenue_copy.groupby('device')['pb_userid_mulD'].mean().reset_index(name = 'mean_pb_missing_userid')

In [ ]:
# plotting
# ios just has a little bit higher percentage of users with multiple devices
ctr_revenue_copy.groupby('device')['pb_userid_mulD'].mean().reset_index(name = 'mean_pb_userid_mulD').plot.bar(x='device')

Plot views/clicks by each problem as box plots

In [ ]:
fig, axes = plt.subplots(nrows = 2, ncols = 4, figsize=(10, 10), sharey = False, sharex = True)
# views
ctr_revenue_copy[['pb_missing_userid', 'views']].boxplot(by = 'pb_missing_userid', ax = axes[0][0])
ctr_revenue_copy[['pb_userid_mix', 'views']].boxplot(by = 'pb_userid_mix', ax = axes[0][1])
ctr_revenue_copy[['pb_deviceid_mulU', 'views']].boxplot(by = 'pb_deviceid_mulU', ax = axes[0][2])
ctr_revenue_copy[['pb_userid_mulD', 'views']].boxplot(by = 'pb_userid_mulD', ax = axes[0][3])
# clicks
ctr_revenue_copy[['pb_missing_userid', 'clicks']].boxplot(by = 'pb_missing_userid', ax = axes[1][0])
ctr_revenue_copy[['pb_userid_mix', 'clicks']].boxplot(by = 'pb_userid_mix', ax = axes[1][1])
ctr_revenue_copy[['pb_deviceid_mulU', 'clicks']].boxplot(by = 'pb_deviceid_mulU', ax = axes[1][2])
ctr_revenue_copy[['pb_userid_mulD', 'clicks']].boxplot(by = 'pb_userid_mulD', ax = axes[1][3])

Does these problems cause differences on clicks?

In [ ]:
for column in ['pb_missing_userid', 'pb_userid_mix', 'pb_deviceid_mulU', 'pb_userid_mulD']:
    display(ctr_revenue_copy.groupby(column)['clicks'].mean().reset_index().rename(columns = {'clicks': 'average_clicks'}))

Mixed users have much less clicks than others. Users with multi devices have less clicks than others

# Start analyzing A/B testing excluding problematic records with metric - CTR

For simplicity, throw away the problematic assignments. In reality, do not do this by default. Need to go through
checks carefully. 

In [ ]:
ctr_revenue_copy_noPb = ctr_revenue_copy[ctr_revenue_copy.pb_all == 0]

In [ ]:
ctr_revenue_copy_noPb.shape

In [ ]:
# what is the date range
print min(ctr_revenue_copy_noPb['date']) # minimum date of dataset
print min(ctr_revenue_copy_noPb['date']) + pd.Timedelta(days=3) # start date of experiment

In [ ]:
# subsetting the data separating by the experiment start timestamp
data_before = ctr_revenue_copy_noPb[ctr_revenue_copy_noPb.date < min(ctr_revenue_copy_noPb['date']) + pd.Timedelta(days=3)]
data_after = ctr_revenue_copy_noPb[ctr_revenue_copy_noPb.date >= min(ctr_revenue_copy_noPb['date']) + pd.Timedelta(days=3)]

In [ ]:
print data_before.shape
print data_after.shape

Before the experiment started, any difference on ctr between treatment and control?

In [ ]:
cVSt_clicks_views_before = data_before.groupby('groups')['clicks','views'].sum()

In [ ]:
cVSt_clicks_views_before['non-clicks'] = cVSt_clicks_views_before['views'] - cVSt_clicks_views_before['clicks']

In [ ]:
cVSt_clicks_views_before = cVSt_clicks_views_before.drop(['views'], axis = 1)

In [ ]:
cVSt_clicks_views_before

In [ ]:
from scipy.stats import chi2_contingency
chi2_contingency(cVSt_clicks_views_before)

Before experiment, the difference on ctr between two groups are not significant

Then, let's check difference on ctr between two groups after experiment starts

In [ ]:
cVSt_clicks_views_after = data_after.groupby('groups')['clicks','views'].sum()

In [ ]:
cVSt_clicks_views_after['non-clicks'] = cVSt_clicks_views_after['views'] - cVSt_clicks_views_after['clicks']

In [ ]:
cVSt_clicks_views_after = cVSt_clicks_views_after.drop(['views'], axis = 1)

In [ ]:
cVSt_clicks_views_after

In [ ]:
from scipy.stats import chi2_contingency
chi2_contingency(cVSt_clicks_views_after)

After experiment, the differnce between two groups are not significant, either.

So, let's deep dive into different countries and different devices to see any difference

In [ ]:
class ztest():
    def ztest_by_subgroup(self, data, bycol, value):
        subset = data[data[bycol] == value]
        subset_clicks_views = subset.groupby('groups')['clicks', 'views'].sum()
        subset_clicks_views['non_clicks'] = subset_clicks_views['views'] - subset_clicks_views['clicks']
        subset_clicks_views = subset_clicks_views.drop(['views'], axis = 1)
        chisq, p, dof, expectedArray = chi2_contingency(subset_clicks_views)
        print "When %s is %s, p-value is %s" %(bycol, value, p)
        return p
    def transform(self, data, bycol):
        unique_vals = data[bycol].unique()
        for value in unique_vals:
            self.ztest_by_subgroup(data, bycol, value)
    

In [ ]:
groupsXdevices = data_after.groupby(['groups', 'device']).size().reset_index(name = 'count').pivot(index = 'groups', columns = 'device')
groupsXdevices

In [ ]:
chi2_contingency(groupsXdevices)

the device counts are imbalanced in two groups

In [ ]:
ztest().transform(data_after, 'device')

IOS has significant difference

In [ ]:
groupsXcountries = data_after.groupby(['groups', 'country']).size().reset_index(name = 'count').pivot(index = 'groups', columns = 'country')
groupsXcountries

In [ ]:
chi2_contingency(groupsXcountries)

In [ ]:
ztest().transform(data_after, 'country')

No specific country has significant difference on CTR

# Start analyzing A/B testing excluding problematic records with metric - Revenue

Next, let's check the revenue after checking CTR

In [ ]:
ctr_revenue_copy.revenue.hist(bins = 100).set_xlim((0,150))

In [ ]:
ctr_revenue_copy[ctr_revenue_copy.revenue == 0].shape[0] * 1.0/ctr_revenue_copy.shape[0]

This is so highly skewed

In [ ]:
# now only consider the revenue > 0
Revenue = ctr_revenue_copy[ctr_revenue_copy.revenue > 0][['revenue']]

In [ ]:
Revenue.revenue.hist(bins = 100).set_xlim((0,900))

There are some extremely high outliers

In [ ]:
Revenue[['revenue']].boxplot()

In [ ]:
Revenue_win = Revenue.copy()

In [ ]:
# Let's do data winsorization
bound = Revenue_win.revenue.quantile(0.99) 
Revenue_win.loc[Revenue_win.revenue > bound, 'revenue'] = bound

In [ ]:
Revenue_win[['revenue']].boxplot().set_ylim((0, 200))

In [ ]:
fig, axes = plt.subplots(nrows = 1, ncols = 2, sharey = True)
Revenue.plot(kind='hist',subplots=True,sharex=True,sharey=True, ax = axes[0])
Revenue_win.plot(kind='hist',subplots=True,sharex=True,sharey=True, ax = axes[1])
axes[0].text(500, 2100,'Revenue', horizontalalignment='center')
axes[1].text(500, 2100,'New Revenue (without 0)', horizontalalignment='center')

Bootstrap - comparing revenue between groups

In [ ]:
E_mean = np.mean(Revenue_win.revenue)
E_var = np.var(Revenue_win.revenue)/Revenue_win.shape[0]
print "The mean of Revenue_win is %s, and variance of Revenue_win is %s" %(E_mean, E_var)

In [ ]:
True = np.mean(Revenue_win.revenue)
btsample = []
for i in range(1000):
    sample = Revenue_win.sample(Revenue_win.shape[0], replace = True)
    btsample.append(np.mean(sample))

In [ ]:
var_bt = np.var(btsample)
var_bt

In [ ]:
E_var

In [ ]:
True = Revenue_win.revenue.quantile(0.75)
btsample = []
for i in range(1000):
    sample = Revenue_win.sample(Revenue_win.shape[0], replace = True)
    btsample.append(sample.revenue.quantile(0.75))

In [ ]:
print Revenue_win.revenue.quantile(0.75)
print btsample

In [ ]:
print np.var(btsample)

# Results analysis

In [ ]:
bound = Revenue.revenue.quantile(0.999)
data_before['revenue_win'] = np.where(data_before['revenue'] > bound, bound, data_before['revenue'])

In [ ]:
data_after['revenue_win'] = np.where(data_after['revenue'] > bound, bound, data_after['revenue'])

In [ ]:
from scipy.stats import ttest_ind

treatment = data_after[data_after['groups']=='treatment']
control = data_after[data_after['groups']=='control']

ttest_ind(treatment['revenue_win'], control['revenue_win'], equal_var = False)

Thus, treatment and control revenue are not significant different after experiment started

There might be bias before experiment started. So, Let's do Regression Adjustment & diff-in-diff analysis

In [ ]:
# trying to do sql in pandas
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
# Average daily revenue after experiment started (11 days)
daily_rev_post = pysqldf("select userid, country, device, groups, sum(revenue_win)/11 as rev_post \
from data_after group by 1,2,3,4")
# Average daily revenue before experiment started (3 days)
daily_rev_pre = pysqldf("select userid, country, device, groups, date, sum(revenue_win)/3 as rev_pre \
                      from data_before group by 1,2,3,4")
# Combine daily_rev_post with daily_rev_pre
daily_rev = pysqldf('select a.*, coalesce(rev_pre,0) as rev_pre from daily_rev_post a left outer join daily_rev_pre b on a.userid = b.userid')


In [ ]:
x1 = daily_rev[daily_rev.groups == 'treatment'].rev_post - daily_rev[daily_rev.groups == 'treatment'].rev_pre
x2 = daily_rev[daily_rev.groups == 'control'].rev_post - daily_rev[daily_rev.groups == 'control'].rev_pre

In [ ]:
ttest_ind(x1, x2, equal_var = False)

So, diff-in-dff are significant between treatment and control group

In [ ]:
from statsmodels.formula.api import ols
mod = ols(formula = 'rev_post ~ groups + country + device + rev_pre', data = daily_rev)
res = mod.fit()
print(res.summary())

No obvious relationship between specific country/device and revenue

# Cohort analysis 

on D4 users, analyzing change over time

In [ ]:
d4_userid = data_after[data_after.date == '2017-05-11'].userid.values
d4 = data_after[data_after['userid'].apply(lambda x: x in set(d4_userid))]

In [ ]:
cohort_df = pd.DataFrame(columns=['date','p.value'])

In [ ]:
cohort_list = []
n_dist_date = len(ctr_revenue.date.unique())
for i in range(n_dist_date - 3):
    print i
    date = np.sort(ctr_revenue.date.unique())[i+3]
    p = ztest().ztest_by_subgroup(d4, 'date', date)
    cohort_list.append((date, p))
cohort_df = pd.DataFrame(cohort_list, columns = ['date', 'p-value'])
cohort_df

In [ ]:
cohort_df.set_index('date').plot()